1. Dataset Exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Files are too large to perform cleaning on all at once
df = pd.read_csv('/home/sagemaker-user/cybersecurity-tensor-ad/data/raw/cicids2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

Shape: 225,745 rows × 79 columns


In [3]:
print(df.columns.tolist())
print("="*80)

print(df.info())

[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag 

In [4]:
# Remove inconsistent column names with extra spaces
df.columns = df.columns.str.strip()

print(df.columns.tolist())

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count

In [5]:
# Columns to be removed
# Including metadata will allow the feature to act as a shortcut predictor
# The distribution of the feature in relation to the label is not controlled

# Training data: SSH attack always on port 22
# Real world: SSH attack on port 2222 custom port
# Result: model misses the attack

# Training data: Port 80 has lots of DDoS samples
# Real world: Port 80 is your busiest legitimate service
# Result: Tons of false positives

# For more information, read the following paper
# D’hooge, L., Verkerken, M., Volckaert, B., Wauters, T., De Turck, F. (2022).
# Establishing the Contaminating Effect of Metadata Feature Inclusion in
# Machine-Learned Network Intrusion Detection Models. In: Cavallaro, L., Gruss,
# D., Pellegrino, G., Giacinto, G. (eds) Detection of Intrusions and Malware,
# and Vulnerability Assessment. DIMVA 2022. Lecture Notes in Computer Science,
# vol 13358. Springer, Cham. https://doi.org/10.1007/978-3-031-09484-2_2
drop_columns = [
    "Flow ID",  # Unique identifier with no predictive value
    "Source IP", "Src IP",  # model would memorize attacker IPs
    "Source Port", "Src Port",  # ephemeral, not behavior-indicative
    "Destination IP", "Dst IP",  # specific targets in simulation
    "Destination Port", "Dst Port",  # attacks target specific ports in dataset
    "Timestamp",  # Model will learn the time frames of the attacks
    'Fwd Header Length.1',  # Duplicate column
]

df.drop(columns=drop_columns, inplace=True, errors="ignore")

In [6]:
# correct columns to the correct dtype
int_col = df.select_dtypes(include='integer').columns
df[int_col] = df[int_col].apply(pd.to_numeric, errors='coerce', downcast='integer')
float_col = df.select_dtypes(include='float').columns
df[float_col] = df[float_col].apply(pd.to_numeric, errors='coerce', downcast='float')

df['Label'].replace({'BENIGN': 'Benign'}, inplace=True)
df['Label'] = df.Label.astype('category')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225745 entries, 0 to 225744
Data columns (total 77 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Flow Duration                225745 non-null  int32   
 1   Total Fwd Packets            225745 non-null  int16   
 2   Total Backward Packets       225745 non-null  int16   
 3   Total Length of Fwd Packets  225745 non-null  int32   
 4   Total Length of Bwd Packets  225745 non-null  int32   
 5   Fwd Packet Length Max        225745 non-null  int16   
 6   Fwd Packet Length Min        225745 non-null  int16   
 7   Fwd Packet Length Mean       225745 non-null  float32 
 8   Fwd Packet Length Std        225745 non-null  float32 
 9   Bwd Packet Length Max        225745 non-null  int16   
 10  Bwd Packet Length Min        225745 non-null  int16   
 11  Bwd Packet Length Mean       225745 non-null  float32 
 12  Bwd Packet Length Std        225745 non-null

In [7]:
# Remove the NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
print(df.isna().any(axis=1).sum(), "rows with at least one NaN to remove")
df.dropna(inplace=True)

34 rows with at least one NaN to remove


In [8]:
# Remove duplicates
df.drop_duplicates(inplace=True, subset=df.columns.difference(['Label']))
print(df["Label"].value_counts())
print(f"shape: {df.shape}\n")

Label
DDoS      128014
Benign     93250
Name: count, dtype: int64
shape: (221264, 77)

